Electricity showcase for basic PULPO

Written by Fabian Lechtenberg, 18.09.2023

Last Update: 23.09.2023

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">(1) Selection of LCI Data</h2>
</div>

### Import section

In this working version of the pulpo repository, pulpo musst be imported from the folder above, which can be done by appending ".." to the system path. Later, only the import after installation via pip will be necessary.

In [1]:
import os
import sys
sys.path.append('../')
from pulpo import pulpo

import pandas as pd
pd.set_option('display.max_colwidth', None)

### Setup

Specify the project, database and method to be used. Also indicate the folder where the working data should be stored. For this example to work, you need a project "**pulpo**" with a database "**cutoff38**", which is the ecoinvent 3.8 cutoff system model.

In [2]:
project = "pulpo"
database = "cutoff38"
methods = "('IPCC 2013', 'climate change', 'GWP 100a')"

# Substitute with your working directory of choice
notebook_dir = os.path.dirname(os.getcwd())
directory = os.path.join(notebook_dir, 'data')

# Substitute with your GAMS path
GAMS_PATH = "C:/GAMS/37/gams.exe"

Create a pulpo object called "pulpo_worker". This object is an element of the class "PulpoOptimizer", a class that links the different utilitiy modules containing code for retrieving, preparing and adjusting the data, preparing and running the optimization problem, as well as saving the results.

In [3]:
pulpo_worker = pulpo.PulpoOptimizer(project, database, methods, directory)

Retrieve the data. If data is already loaded, this step is automatically skipped. 

In [4]:
pulpo_worker.get_lci_data()

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">(2) User Specifications</h2>
</div>

<div style="text-align: center;">
    <img src="pictures/electricity_showcase_1.png" alt="Image Alt Text" style="width: 50%;" />
</div>

### Specify the **functional unit**

Retrieve the market activity for liquid hydrogen in Europe (RER). Use the function "**<span style="color: red;">retrieve_activities</span>**" for this purpose. The function takes 4 optional arguments: "keys" (🔑) --> "activities" (⚙️) --> "reference_products" (📦) --> "locations" (🗺️). The activities are retrieved by this order. 

Since the key is unique, a single activity for each passed key will be returned. Activity names, reference_prduct and locations are not unique, so the best match for the passed data will be returned. 

#### Passing keys  🔑

Keys can be obtained e.g. directly from **activity browser** and several keys can be passed at the same time.

In [5]:
keys = ["('cutoff38', '962727b9a36bcaa186f222b29b57f6a3')", "('cutoff38', '473d4bb488e8f903b58203f3e5161636')"]

In [6]:
pulpo_worker.retrieve_activities(keys=keys)

['market for electricity, high voltage' (kilowatt hour, DE, None),
 'market for electricity, high voltage' (kilowatt hour, ES, None)]

#### Passing activity  name (⚙️), reference_product (📦) and/or location (🗺️)

Instead of passing the keys, a combination of activities, reference_products and locations can be passed. A best match (all existing combinations) will be returned. 

In [7]:
activities = ["market for electricity, high voltage"]
reference_products = ["electricity, high voltage"]
locations = ["DE"]

It is also possible to pass only partial information such as only reference product or only activity name:

In [8]:
pulpo_worker.retrieve_activities(activities=activities)

['market for electricity, high voltage' (kilowatt hour, IS, None),
 'market for electricity, high voltage' (kilowatt hour, IL, None),
 'market for electricity, high voltage' (kilowatt hour, PT, None),
 'market for electricity, high voltage' (kilowatt hour, US-RFC, None),
 'market for electricity, high voltage' (kilowatt hour, DK, None),
 'market for electricity, high voltage' (kilowatt hour, BD, None),
 'market for electricity, high voltage' (kilowatt hour, CN-SGCC, None),
 'market for electricity, high voltage' (kilowatt hour, US-MRO, None),
 'market for electricity, high voltage' (kilowatt hour, TZ, None),
 'market for electricity, high voltage' (kilowatt hour, MA, None),
 'market for electricity, high voltage' (kilowatt hour, BG, None),
 'market for electricity, high voltage' (kilowatt hour, AT, None),
 'market for electricity, high voltage' (kilowatt hour, TG, None),
 'market for electricity, high voltage' (kilowatt hour, ET, None),
 'market for electricity, high voltage' (kilowatt

In [9]:
pulpo_worker.retrieve_activities(reference_products=reference_products)

['heat and power co-generation, biogas, gas engine' (kilowatt hour, TW, None),
 'treatment of blast furnace gas, in power plant' (kilowatt hour, NL, None),
 'market for electricity, high voltage' (kilowatt hour, UZ, None),
 'electricity production, oil' (kilowatt hour, RU, None),
 'electricity production, hydro, reservoir, alpine region' (kilowatt hour, CA-BC, None),
 'electricity production, compressed air energy storage, adiabatic' (kilowatt hour, RoW, None),
 'electricity production, hard coal' (kilowatt hour, BR-North-eastern grid, None),
 'electricity production, deep geothermal' (kilowatt hour, IT, None),
 'electricity production, wind, 1-3MW turbine, onshore' (kilowatt hour, CA-BC, None),
 'market for electricity, high voltage' (kilowatt hour, MY, None),
 'electricity, high voltage, production mix' (kilowatt hour, CA-AB, None),
 'electricity production, peat' (kilowatt hour, IE, None),
 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilo

Let's retrieve the activity of our functional unit and specify the demand as a dictionary:

In [10]:
electricity_market = pulpo_worker.retrieve_activities(activities=activities, reference_products=reference_products, locations=locations)

In [11]:
electricity_market

['market for electricity, high voltage' (kilowatt hour, DE, None)]

Setting a demand of 128,819.0 GWh of electricity according to [Germany electricity demand 2018](https://www.destatis.de/EN/Themes/Society-Environment/Environment/Material-Energy-Flows/Tables/electricity-consumption-households.html)


In [12]:
demand = {electricity_market[0]: 1.28819e+11}

### Specify the **choices**

The choices are specified similar to the demand / functional unit. First, search for the processes with equivalent products:

In [13]:
activities = ["electricity production, lignite", 
             "electricity production, hard coal",
             "electricity production, nuclear, pressure water reactor",
             "electricity production, wind, 1-3MW turbine, onshore"]
reference_products = ["electricity, high voltage"]
locations = ["DE"]

electricity_activities = pulpo_worker.retrieve_activities(activities=activities, reference_products=reference_products, locations=locations)

In [14]:
electricity_activities

['electricity production, lignite' (kilowatt hour, DE, None),
 'electricity production, nuclear, pressure water reactor' (kilowatt hour, DE, None),
 'electricity production, hard coal' (kilowatt hour, DE, None),
 'electricity production, wind, 1-3MW turbine, onshore' (kilowatt hour, DE, None)]

These are the currently four most employed technologies for electricity production in Germany (lignite: 24.2%, wind: 15.4%, coal: 11.9%, nuclear: 10.6%) according to the "_market for electricity, high voltage (DE)_"

Specify also the choices as a dictionary. Be aware, that this time we are dealing with a dictionary of dictionaries. Each inner dictionary corresponds to one type of choice in the background! Here, we only consider choices between electricity production activities, so we assign the key "electricity" to the equivalent product they produce. 

The assigned value in the inner dictionary is the capacity limit of this activity, which for now is set to a very high value, to consider an unconstrained situation. 

In [15]:
choices  = {'electricity': {electricity_activities[0]: 1e16,
                         electricity_activities[1]: 1e16,
                         electricity_activities[2]: 1e16,
                         electricity_activities[3]: 1e16}}

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">(3) Solution</h2>
</div>

### Instantiate the worker

In [16]:
instance = pulpo_worker.instantiate(choices=choices, demand=demand)

Creating Instance
Instance created


### Solve the instance

When specifying a valid GAMS_PATH with a licence for CPLEX, as shown below, CPLEX with fine-tuned parameters is automatically selected to solve the Linear Problem (**LP**).

If no GAMS_PATH is specified, the "[HiGHS](https://highs.dev/)" solver is automatically used. It has almost double the run time of "CPLEX".

In [ ]:
#results = pulpo_worker.solve()
# Alternatively using GAMS (cplex) solvers:
results = pulpo_worker.solve(GAMS_PATH=GAMS_PATH)

GAMS solvers library availability: True
Solver path: C:\GAMS\37\gams.exe
        2.40 seconds required for presolve
--- Job model.gms Start 12/11/23 15:38:17 37.1.0 r07954d5 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\GAMS\37\gmsprmNT.txt
    C:\Users\Usuario\Documents\GAMS\gamsconfig.yaml
--- GAMS Parameters defined
    MIP CPLEX
    Input C:\Users\Usuario\AppData\Local\Temp\tmph4tvxzba\model.gms
    Output C:\Users\Usuario\AppData\Local\Temp\tmph4tvxzba\output.lst
    ScrDir C:\Users\Usuario\AppData\Local\Temp\tmph4tvxzba\225a\
    SysDir C:\GAMS\37\
    CurDir C:\Users\Usuario\AppData\Local\Temp\tmph4tvxzba\
    LogOption 3
    OptCR 1E-9
Licensee: Antonio Espuna, Single User License            S210319|0002AN-WIN
          Universitat Politecnica de Catalunya, Chemical Engineering DC6757
          C:\Users\Usuario\Documents\GAMS\gamslice.txt
          antonio.espuna@upc.edu                                           
Processor information: 1 socket(s), 12 core(s), and 20 thread

### Save and summarize the results 💾📈

The "**save_results()**" function will save the results in an processed format to an excel file in the data folder that has been specified at the beginning.

In [ ]:
pulpo_worker.save_results(choices=choices, demand=demand, name='electricity_showcase_results.xlsx')

You can inspect the generated excel file.

There is another function to summarize the results in dataframe form within jupyter notbeooks called "summarize_results". This function has similar inputs to the "save_results" function, but does not require the specification of a filename. Additionally, by specifying the "*zeroes*" parameter to "*True*" all the not-selected choices are omitted in the summary.

In [ ]:
pulpo_worker.summarize_results(choices=choices, demand=demand, zeroes=True)

# Closing Remarks

This is the end of the very basic PULPO functionalities using the electricity case study. 

The following sections will dive deeper into additional functionalities.

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">Additional Constraints</h2>
</div>

<div style="text-align: center;">
    <img src="pictures/electricity_showcase_2.png" alt="Image Alt Text" style="width: 50%;" />
</div>

Let's assess what happens if the "_electricity production, nuclear, pressure water reactor | electricity, high voltage | DE_" activity is indirectly constrained trough a restriction on "_nuclear fuel element, for pressure water reactor, UO2 4.0% & MOX_"

In [ ]:
activities = ["market for nuclear fuel element, for pressure water reactor, UO2 4.0% & MOX"]
reference_products = ["nuclear fuel element, for pressure water reactor, UO2 4.0% & MOX"]
locations = ["GLO"]

nuclear_fuel = pulpo_worker.retrieve_activities(activities=activities, reference_products=reference_products, locations=locations)

In [ ]:
nuclear_fuel

In [ ]:
upper_limit = {nuclear_fuel[0]: 100000}

The rationale behind choosing this activity and this limit is based on inspection of the scaling vector of the previous results. This activity is limiting for the nuclear electricity activity but not for the others, so, to enforce a different result than before, this activity is constrained.

In [ ]:
pulpo_worker.instantiate(choices=choices, demand=demand, upper_limit=upper_limit)
results = pulpo_worker.solve()

In [ ]:
pulpo_worker.summarize_results(choices=choices, demand=demand, constraints=upper_limit)

As can be seen from the summary above, part of the final electricity demand is supplied by the wind turbine processes, because the nuclear process is constrained by the nuclear fuel process. It is also evident that the impact is higher than the previous one, as the GWP minimizing process (nuclear) can no longer supply the full demand.

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">Additional Methods</h2>
</div>

Let's see how to evaluate different methods and set them as objectives, in this case evaluating the ReCiPe endpoints, and setting the human health one as objective:

In [ ]:
methods = {"('IPCC 2013', 'climate change', 'GWP 100a')": 0,
           "('ReCiPe Endpoint (E,A)', 'resources', 'total')": 0,
           "('ReCiPe Endpoint (E,A)', 'human health', 'total')": 1,
           "('ReCiPe Endpoint (E,A)', 'ecosystem quality', 'total')": 0}

With this, a new Pulpo worker must be created:

In [ ]:
pulpo_worker = pulpo.PulpoOptimizer(project, database, methods, directory)

In [ ]:
pulpo_worker.get_lci_data()

In [ ]:
pulpo_worker.instantiate(choices=choices, demand=demand)
results = pulpo_worker.solve()

In [ ]:
pulpo_worker.summarize_results(choices=choices, demand=demand, zeroes=True)

From the summary above, it can be seen that for the "_human health_" category, the nuclear process is not the most suitable anymore. With this objective, the wind turbine process is selected. 

As another category is minimized, the GWP has changed as well: previously, with nuclear electricity the total GWP was 1.599836e+10 while with wind electricity it is 1.739234e+10.

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">Lower Level Decisions</h2>
</div>

<div style="text-align: center;">
    <img src="pictures/electricity_showcase_3.png" alt="Image Alt Text" style="width: 50%;" />
</div>

In this case study, we would like to keep the current share of the electricity supplied by fossil sources the same. The choices that we consider on the electricity production level are between the coal and lignite activities:

In [ ]:
methods = {"('IPCC 2013', 'climate change', 'GWP 100a')": 1,
           "('ReCiPe Endpoint (E,A)', 'resources', 'total')": 0,
           "('ReCiPe Endpoint (E,A)', 'human health', 'total')": 0,
           "('ReCiPe Endpoint (E,A)', 'ecosystem quality', 'total')": 0}

In [ ]:
pulpo_worker = pulpo.PulpoOptimizer(project, database, methods, directory)
pulpo_worker.get_lci_data()

In [ ]:
activities = ["electricity production, lignite", 
             "electricity production, hard coal"]
reference_products = ["electricity, high voltage"]
locations = ["DE"]

electricity_activities = pulpo_worker.retrieve_activities(activities=activities, reference_products=reference_products, locations=locations)

Instead of assessing only the **technology** choices, we are invetigating the best **regional** choice for the source of coal and lignite:

In [ ]:
coal_activities = ["market for hard coal"]
lignite_activities = ["market for lignite"]

coal_activities = pulpo_worker.retrieve_activities(activities=coal_activities)
lignite_activities = pulpo_worker.retrieve_activities(activities=lignite_activities)

The updated choice dictionary looks like this:

In [ ]:
choices  = {'electricity': {elec: 1e16 for elec in electricity_activities},
            'coal': {coal: 1e16 for coal in coal_activities},
            'lignite': {lignite: 1e16 for lignite in lignite_activities}}

Instantiating and solving the adapted problem:

In [ ]:
pulpo_worker.instantiate(choices=choices, demand=demand)
results = pulpo_worker.solve()

Visualizing the results

In [ ]:
pulpo_worker.summarize_results(choices=choices, demand=demand, zeroes=True)

I can be seen that out of the fossil alternatives, the electricity from coal minimizes GWP when the coal comes from RLA [Latin America and the Caribbean] (omitting transport emissions). Moreover, it can be seen that the market for lignite is somewhere used upstream of the coal production activity. In the place where it is used, the RER market for lignite is the preferred one. The lignite consumption is one order of magnitude lower than the coal consumption.

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">Supply vs. Demand Problem</h2>
</div>

Finally, let's test and assess the functionality of PULPO to specify supply values rather than demand values. This can be done by setting the lower_limit and the upper_limit of activities to the same value. This will enforce the corresponding scaling vector entry of that activity to the specified value, and activates the slack variable to relax the demand value. 

This can simply be done by specifying the upper and lower limits rather than the demand (note, we continue with the choices from the previous section):

In [ ]:
upper_limit = {electricity_market[0]: 1.28819e+11}
lower_limit = {electricity_market[0]: 1.28819e+11}

In [ ]:
pulpo_worker.instantiate(choices=choices, upper_limit=upper_limit, lower_limit=lower_limit)
results = pulpo_worker.solve()

In [ ]:
pulpo_worker.summarize_results(choices=choices, demand=demand, constraints=upper_limit, zeroes=True)

From the results it can be observed that the resulting GWP is **considerably** lower (6.191744e+10 vs. 5.920967e+10: ~4.4%) than in the previous section. Now, the production value (supply) of electricity is specified, so that electricity consumed in the background is accounted for in the specifications.

Overall, when specifying supply values instead of demand values, the corresponding scaling vector entries are always smaller.

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">Foreground vs. Background Modelling</h2>
</div>

<div style="text-align: center;">
    <img src="pictures/electricity_showcase_4.png" alt="Image Alt Text" style="width: 50%;" />
</div>

In this final part of the electricity showcase, the difference between foreground and background modelling and optimization is demonstrated. For that purpose a foreground system must be created ... this can be done either by hand or I can upload a folded foreground system for this case ... in essence, a new database must be created (e.g. "_cutoff38_foreground_" where copies from the market and electricity production activites are duplicated to from the "_cutoff38_". What this does is disconnecting this activities from their downstream.

We are then replacing the original inputs in the electricity market with the duplicated processes from the foreground system, effectively disconnecting the choices made in the electricity market from the upstream of the electricity production technologies:

With this in mind, we can set up the optimization as usual, importing the "_cutoff38_foreground_" database instead of the "_cutoff38_" database:

In [ ]:
project = "pulpo"
database = "cutoff38_foreground"

In [ ]:
pulpo_worker = pulpo.PulpoOptimizer(project, database, methods, directory)

In [ ]:
pulpo_worker.get_lci_data()

Now, choose the foreground market for electricity:

In [ ]:
activities = ["market for electricity, high voltage, foreground"]
reference_products = ["electricity, high voltage"]
locations = ["DE"]

In [ ]:
foreground_market = pulpo_worker.retrieve_activities(activities=activities, reference_products=reference_products, locations=locations)

In [ ]:
demand = {foreground_market[0]: 1.28819e+11}

And the foreground electricity production technologies:

In [ ]:
activities = ["electricity production, lignite, foreground", 
             "electricity production, hard coal, foreground",
             "electricity production, nuclear, pressure water reactor, foreground",
             "electricity production, wind, 1-3MW turbine, onshore, foreground"]
reference_products = ["electricity, high voltage"]
locations = ["DE"]

electricity_activities = pulpo_worker.retrieve_activities(activities=activities, reference_products=reference_products, locations=locations)

In [ ]:
choices  = {'electricity': {electricity_activities[0]: 1e16,
                         electricity_activities[1]: 1e16,
                         electricity_activities[2]: 1e16,
                         electricity_activities[3]: 1e16}}

Create the instance and solve the problem:

In [ ]:
instance = pulpo_worker.instantiate(choices=choices, demand=demand)

In [ ]:
results = pulpo_worker.solve()

In [ ]:
pulpo_worker.summarize_results(choices=choices, demand=demand, zeroes=True)

The result of this optimization is a minimum GWP of 1.764977e+10 kg CO2eq instead of 1.599836e+10 kg CO2eq from the initial calculations uising the full LCI. This is a difference of **8%**!

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">Integer Cuts</h2>
</div>

This functionality will be available in future versions of PULPO